In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd
import time

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

NameError: name 'auth' is not defined

In [3]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [4]:
%%time
infile = open('./data/1000_Y_dgp_tim.p', 'rb')
Y_dgp = pickle.load(infile)
# Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape, '\n',Y_dgp)

(2798, 1000) 
 [[3. 3. 0. ... 0. 1. 2.]
 [3. 3. 1. ... 0. 1. 3.]
 [1. 1. 1. ... 0. 3. 1.]
 ...
 [3. 3. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 1.]
 [2. 2. 1. ... 2. 1. 2.]]
Wall time: 28.9 ms


In [11]:
%%time
m = [25]
for draws in m:
    t=time.time()
    result = []
    start = 0
    end = 200
    method = 'SMC'
    skips = 0
    for i in range(start,end):
        print('iteration: %i' %(i+1))
        try:
            qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
            solution = qmc.solver()
            result.append(solution)
        except np.linalg.LinAlgError:
            skips += 1
            print('deze is overgeslagen, optimizer doet naar, %i skips'%(skips))
        if len(result)== 10:
            pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_dgp_results.p" %(method,draws,i-9,i,method,draws), "wb" ) )
            result = []
            skips = 0
    print('%s(%i) run time: %f'%(method,draws,time.time()-t))
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 1
      fun: 2872.2918465141997
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00090949, -0.00300133,  0.01405169,  0.00300133, -0.00309228,
        0.00040927])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 266
      nit: 30
   status: 0
  success: True
        x: array([ 1.56092346,  1.1399581 , -1.08425938, -7.14835064, -6.2029136 ,
       -0.54770589]) 
 11.29736065864563
iteration: 2
      fun: 2835.04263383933
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00741238,  0.00236469,  0.04029062,  0.00140972,  0.        ,
       -0.00422915])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 329
      nit: 36
   status: 0
  success: True
        x: array([ 1.48408286,  0.99518922, -1.12932857, -1.34545956, -8.42237985,
       -0.68425279]) 
 13.725349426269531
iteration: 3
      fun: 2894.126230597938
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac